In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

### Reading in all data while parsing dates

In [3]:
transactions_full = pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])
# transactions_train_og = transactions_train_og.set_index("TX_ID")

merchants_full = pd.read_csv('./data/merchants.csv', parse_dates=["FOUNDATION_DATE","ACTIVE_FROM","TRADING_FROM"])

terminals_full = pd.read_csv('./data/terminals.csv')

customers_full = pd.read_csv('./data/customers.csv')

C:\Users\joshu\AppData\Local\Temp\ipykernel_24868\2224522130.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_full = pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])


### Parsing the date and adding 2 new columns with just the date and time of the transaction

In [4]:
# transactions_full["CARD_EXPIRY_DATE"] = transactions_full["CARD_EXPIRY_DATE"].apply(lambda x: parse(x))
transactions_full["TX_TS"] = pd.to_datetime(transactions_full["TX_TS"],format='%Y-%m-%d')
# transactions_full["TX_TS_DATE"] = pd.to_datetime(transactions_full["TX_TS"].dt.date)
# transactions_full["TX_TS_TIME"] = pd.to_datetime(transactions_full["TX_TS"].dt.time)

### Merging datasets

For now I am leaving out x and y coordinates of customer and terminals

In [5]:
transactions_full = pd.merge(transactions_full,merchants_full,on='MERCHANT_ID')

In [6]:
transactions_full.head()

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,CARD_EXPIRY_DATE,CARD_DATA,...,AVERAGE_TICKET_SALE_AMOUNT,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_REQUIRED_PERCENTAGE,DEPOSIT_PERCENTAGE,DELIVERY_SAME_DAYS_PERCENTAGE,DELIVERY_WEEK_ONE_PERCENTAGE,DELIVERY_WEEK_TWO_PERCENTAGE,DELIVERY_OVER_TWO_WEEKS_PERCENTAGE
0,c6dde46458f3d1cfeb9256f8add45a14fb349f8f,2021-04-02 12:42:24+00:00,7323088226725338,55875360,0.00,0,0.00,0.0,12/22,5428********818,...,59,37,63,0,69,69,24,20,20,36
1,10f578bdefbdfb64e222e0e566ac2885da06772f,2021-05-23 22:39:49+00:00,4262297761961723,15873857,21.80,0,21.80,0.0,03/23,4024********280,...,59,37,63,0,69,69,24,20,20,36
2,2ffa5d887b3a4d3662a0ef993e90fb5aff44b273,2021-09-20 19:22:32+00:00,714771602509423,85599276,37.85,0,37.85,0.0,09/22,4929********218,...,59,37,63,0,69,69,24,20,20,36
3,47682f7be475bd903b94a251ba4738dcf7c9bbb8,2021-11-26 20:29:55+00:00,9867755465717758,92968204,6.90,0,6.90,0.0,05/22,4916********744,...,59,37,63,0,69,69,24,20,20,36
4,450871a09218813b7128e540c8660421945a1738,2021-06-21 19:08:14+00:00,3987679533084498,78424763,57.25,0,57.25,0.0,05/22,5344********877,...,59,37,63,0,69,69,24,20,20,36


Looking if there are any transaction where the tx_tx later than CARD_EXPIRY_DATE

### Deleting columns I don't think will help

In [7]:
transactions_full.dtypes

TX_ID                                                 object
TX_TS                                    datetime64[ns, UTC]
CUSTOMER_ID                                            int64
TERMINAL_ID                                            int64
TX_AMOUNT                                            float64
TX_FRAUD                                               int64
TRANSACTION_GOODS_AND_SERVICES_AMOUNT                float64
TRANSACTION_CASHBACK_AMOUNT                          float64
CARD_EXPIRY_DATE                                      object
CARD_DATA                                             object
CARD_BRAND                                            object
TRANSACTION_TYPE                                      object
TRANSACTION_STATUS                                    object
FAILURE_CODE                                          object
FAILURE_REASON                                        object
TRANSACTION_CURRENCY                                  object
CARD_COUNTRY_CODE       

In [8]:
useless_columns = ['CARD_EXPIRY_DATE','TX_ID','CARD_DATA','LEGAL_NAME','FOUNDATION_DATE','ACTIVE_FROM','TRADING_FROM']
transactions_full.drop(useless_columns,axis=1,inplace=True)

### Dealing with missing data

In [9]:
transactions_full.isna().sum()

TX_TS                                          0
CUSTOMER_ID                                    0
TERMINAL_ID                                    0
TX_AMOUNT                                      0
TX_FRAUD                                       0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT          0
TRANSACTION_CASHBACK_AMOUNT                    0
CARD_BRAND                                     0
TRANSACTION_TYPE                               0
TRANSACTION_STATUS                             0
FAILURE_CODE                             1049208
FAILURE_REASON                           1049208
TRANSACTION_CURRENCY                           0
CARD_COUNTRY_CODE                              0
MERCHANT_ID                                    0
IS_RECURRING_TRANSACTION                       0
ACQUIRER_ID                                    1
CARDHOLDER_AUTH_METHOD                         1
BUSINESS_TYPE                                  0
MCC_CODE                                       0
TAX_EXCEMPT_INDICATO

After doing analysis I saw that the missing Failure_Code and Failure_Reason values are when the transaction was successful so do not want ot drop those. Instead I am going to create a new code and reason for when successful, to remove the NAN.

These transactions were interesting, completed successfully, but under failure reason and were fraud

### 99 will mean successful transaction

In [10]:
transactions_full.loc[transactions_full['FAILURE_CODE'].isna(),['FAILURE_CODE']] = 99

### The corresponding columns will have non failure

In [11]:
transactions_full.loc[transactions_full['FAILURE_REASON'].isna(),['FAILURE_REASON']] = 'Non Failure'

In [12]:
transactions_full.isna().sum()

TX_TS                                    0
CUSTOMER_ID                              0
TERMINAL_ID                              0
TX_AMOUNT                                0
TX_FRAUD                                 0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT    0
TRANSACTION_CASHBACK_AMOUNT              0
CARD_BRAND                               0
TRANSACTION_TYPE                         0
TRANSACTION_STATUS                       0
FAILURE_CODE                             0
FAILURE_REASON                           0
TRANSACTION_CURRENCY                     0
CARD_COUNTRY_CODE                        0
MERCHANT_ID                              0
IS_RECURRING_TRANSACTION                 0
ACQUIRER_ID                              1
CARDHOLDER_AUTH_METHOD                   1
BUSINESS_TYPE                            0
MCC_CODE                                 0
TAX_EXCEMPT_INDICATOR                    0
OUTLET_TYPE                              0
ANNUAL_TURNOVER_CARD                     0
ANNUAL_TURN

### Dropping these rows with missing values as there are only 2

In [13]:
transactions_full.dropna(subset=['ACQUIRER_ID','CARDHOLDER_AUTH_METHOD'], inplace=True)

In [14]:
transactions_full.isna().sum()

TX_TS                                    0
CUSTOMER_ID                              0
TERMINAL_ID                              0
TX_AMOUNT                                0
TX_FRAUD                                 0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT    0
TRANSACTION_CASHBACK_AMOUNT              0
CARD_BRAND                               0
TRANSACTION_TYPE                         0
TRANSACTION_STATUS                       0
FAILURE_CODE                             0
FAILURE_REASON                           0
TRANSACTION_CURRENCY                     0
CARD_COUNTRY_CODE                        0
MERCHANT_ID                              0
IS_RECURRING_TRANSACTION                 0
ACQUIRER_ID                              0
CARDHOLDER_AUTH_METHOD                   0
BUSINESS_TYPE                            0
MCC_CODE                                 0
TAX_EXCEMPT_INDICATOR                    0
OUTLET_TYPE                              0
ANNUAL_TURNOVER_CARD                     0
ANNUAL_TURN

## Missing values dealt with

## Feature Engineering

In [15]:
transactions_full['ANNUAL_TURNOVER_CASH'] = transactions_full['ANNUAL_TURNOVER'] - transactions_full['ANNUAL_TURNOVER_CARD']

#### Binning the time values

In [16]:
transactions_full.iloc[1].TX_TS

Timestamp('2021-05-23 22:39:49+0000', tz='UTC')

In [17]:
bins = [0, 6, 12, 18, 24]
labels = ['00:00-05:59', '06:00-11:59', '12:00-17:59', '18:00-23:59']
transactions_full['Time_of_day'] = pd.cut(transactions_full.TX_TS.dt.hour, bins, labels=labels, right=False)

In [18]:
bins = np.arange(12)
transactions_full['Month'] = transactions_full['TX_TS'].apply(lambda x: x.month)

In [19]:
transactions_full['Day_of_week'] = transactions_full['TX_TS'].apply(lambda x: x.weekday())

In [20]:
transactions_full.drop(['TX_TS'],axis=1,inplace=True)

## Creating a pipeline to deal with categorical data

#### IS_RECURRING_TRANSACTION has this weird behaviour I needed to fix

In [21]:
transactions_full['IS_RECURRING_TRANSACTION'].value_counts()

False    948492
True     100084
False     16117
True       1746
Name: IS_RECURRING_TRANSACTION, dtype: int64

In [22]:
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == 'True',['IS_RECURRING_TRANSACTION']] = 1
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == True,['IS_RECURRING_TRANSACTION']] = 1
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == 'False',['IS_RECURRING_TRANSACTION']] = 0
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == False,['IS_RECURRING_TRANSACTION']] = 0

transactions_full['IS_RECURRING_TRANSACTION'] = transactions_full['IS_RECURRING_TRANSACTION'].apply(pd.to_numeric)

In [23]:
transactions_full.dtypes

CUSTOMER_ID                                 int64
TERMINAL_ID                                 int64
TX_AMOUNT                                 float64
TX_FRAUD                                    int64
TRANSACTION_GOODS_AND_SERVICES_AMOUNT     float64
TRANSACTION_CASHBACK_AMOUNT               float64
CARD_BRAND                                 object
TRANSACTION_TYPE                           object
TRANSACTION_STATUS                         object
FAILURE_CODE                               object
FAILURE_REASON                             object
TRANSACTION_CURRENCY                       object
CARD_COUNTRY_CODE                          object
MERCHANT_ID                                object
IS_RECURRING_TRANSACTION                    int64
ACQUIRER_ID                                object
CARDHOLDER_AUTH_METHOD                     object
BUSINESS_TYPE                              object
MCC_CODE                                    int64
TAX_EXCEMPT_INDICATOR                        bool


In [24]:
[transactions_full.dtypes == 'object']

[CUSTOMER_ID                              False
 TERMINAL_ID                              False
 TX_AMOUNT                                False
 TX_FRAUD                                 False
 TRANSACTION_GOODS_AND_SERVICES_AMOUNT    False
 TRANSACTION_CASHBACK_AMOUNT              False
 CARD_BRAND                                True
 TRANSACTION_TYPE                          True
 TRANSACTION_STATUS                        True
 FAILURE_CODE                              True
 FAILURE_REASON                            True
 TRANSACTION_CURRENCY                      True
 CARD_COUNTRY_CODE                         True
 MERCHANT_ID                               True
 IS_RECURRING_TRANSACTION                 False
 ACQUIRER_ID                               True
 CARDHOLDER_AUTH_METHOD                    True
 BUSINESS_TYPE                             True
 MCC_CODE                                 False
 TAX_EXCEMPT_INDICATOR                    False
 OUTLET_TYPE                            

In [25]:
one_hot_encoded_col = ['CARD_BRAND','TRANSACTION_TYPE','TRANSACTION_STATUS','FAILURE_CODE','FAILURE_REASON','TRANSACTION_CURRENCY','CARD_COUNTRY_CODE','ACQUIRER_ID','CARDHOLDER_AUTH_METHOD','BUSINESS_TYPE','OUTLET_TYPE','Time_of_day']
transactions_full = pd.get_dummies(transactions_full, columns = one_hot_encoded_col)

In [26]:
transactions_full.columns[[transactions_full.dtypes == 'object']]

c:\Users\joshu\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexes\base.py:5055: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Index(['MERCHANT_ID'], dtype='object')

In [27]:
ordinal_encoder = OrdinalEncoder()
transactions_full['MERCHANT_ID'] = ordinal_encoder.fit_transform(transactions_full['MERCHANT_ID'].to_numpy().reshape(-1, 1))

In [28]:
transactions_full.head()

,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,MCC_CODE,TAX_EXCEMPT_INDICATOR,...,BUSINESS_TYPE_Limited Liability Company (LLC),BUSINESS_TYPE_S Corporations,BUSINESS_TYPE_Sole Proprietorships,OUTLET_TYPE_Ecommerce,OUTLET_TYPE_Face to Face,OUTLET_TYPE_Face to Face and Ecommerce,Time_of_day_00:00-05:59,Time_of_day_06:00-11:59,Time_of_day_12:00-17:59,Time_of_day_18:00-23:59
0,7323088226725338,55875360,0.00,0,0.00,0.0,2294.0,0,6533,False,...,0,0,1,0,1,0,0,0,1,0
1,4262297761961723,15873857,21.80,0,21.80,0.0,2294.0,0,6533,False,...,0,0,1,0,1,0,0,0,0,1
2,714771602509423,85599276,37.85,0,37.85,0.0,2294.0,0,6533,False,...,0,0,1,0,1,0,0,0,0,1
3,9867755465717758,92968204,6.90,0,6.90,0.0,2294.0,0,6533,False,...,0,0,1,0,1,0,0,0,0,1
4,3987679533084498,78424763,57.25,0,57.25,0.0,2294.0,0,6533,False,...,0,0,1,0,1,0,0,0,0,1


In [29]:
transactions_full.count()

CUSTOMER_ID                               1066439
TERMINAL_ID                               1066439
TX_AMOUNT                                 1066439
TX_FRAUD                                  1066439
TRANSACTION_GOODS_AND_SERVICES_AMOUNT     1066439
                                           ...   
OUTLET_TYPE_Face to Face and Ecommerce    1066439
Time_of_day_00:00-05:59                   1066439
Time_of_day_06:00-11:59                   1066439
Time_of_day_12:00-17:59                   1066439
Time_of_day_18:00-23:59                   1066439
Length: 124, dtype: int64

In [32]:
print(transactions_full.dtypes)

CUSTOMER_ID                                 int64
TERMINAL_ID                                 int64
TX_AMOUNT                                 float64
TX_FRAUD                                    int64
TRANSACTION_GOODS_AND_SERVICES_AMOUNT     float64
                                           ...   
OUTLET_TYPE_Face to Face and Ecommerce      uint8
Time_of_day_00:00-05:59                     uint8
Time_of_day_06:00-11:59                     uint8
Time_of_day_12:00-17:59                     uint8
Time_of_day_18:00-23:59                     uint8
Length: 124, dtype: object


In [30]:
# Divide data into training and validation subsets
y = transactions_full['TX_FRAUD']
X = transactions_full.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

In [31]:
X_train_full.to_csv('./data/X_train_full.csv',index=False)
X_valid_full.to_csv('./data/X_valid_full.csv',index=False)
y_train.to_csv('./data/y_train.csv',index=False)
y_valid.to_csv('./data/y_valid.csv',index=False)